In [17]:
import json
from pprint import pprint

import requests
import pandas as pd
from sqlalchemy import text

from detroit_tech_employment import db_engine
from detroit_tech_employment.auth import refresh_auth
from detroit_tech_employment.reference import DETROIT_ZIPS

In [5]:
top_lev_codes_q = text("""
select *
from naics.codes
where length(code) = 2;
""")


with db_engine.connect() as db:
    result = db.execute(top_lev_codes_q)

In [6]:
def convert_to_df(response):
    fields =  response["data"]
    return pd.DataFrame({field["name"]: field["rows"] for field in fields})

In [62]:
auth = refresh_auth()
headers = {
    "Authorization": f"Bearer {auth['access_token']}",
    "Content-Type": "application/json"
}

In [8]:

url = "https://agnitio.emsicloud.com/emsi.us.industry/2024.2"

payload = {

    "metrics": [
        {
            "name": "Jobs.2012",
            "as": "2012 Jobs",
        }, 
        {
            "name": "Jobs.2013",
            "as": "2013 Jobs",
        }, 
        {
            "name": "Jobs.2014",
            "as": "2014 Jobs",
        }, 
        {
            "name": "Jobs.2015",
            "as": "2015 Jobs",
        }, 
        {
            "name": "Jobs.2016",
            "as": "2016 Jobs",
        }, 
        {
            "name": "Jobs.2017",
            "as": "2017 Jobs",
        }, 
        {
            "name": "Jobs.2018",
            "as": "2018 Jobs",
        }, 
        {
            "name": "Jobs.2019",
            "as": "2019 Jobs",
        }, 
        {
            "name": "Jobs.2020",
            "as": "2020 Jobs",
        }, 
        {
            "name": "Jobs.2021",
            "as": "2021 Jobs",
        }, 
        {
            "name": "Jobs.2022",
            "as": "2022 Jobs",
        }, 
        {
            "name": "Jobs.2023",
            "as": "2023 Jobs",
        }, 
        {
            "name": "Earnings.2013",
            "as": "2013 Earnings",
        }, 
        {
            "name": "Earnings.2014",
            "as": "2014 Earnings",
        }, 
        {
            "name": "Earnings.2015",
            "as": "2015 Earnings",
        }, 
        {
            "name": "Earnings.2016",
            "as": "2016 Earnings",
        }, 
        {
            "name": "Earnings.2017",
            "as": "2017 Earnings",
        }, 
        {
            "name": "Earnings.2018",
            "as": "2018 Earnings",
        }, 
        {
            "name": "Earnings.2019",
            "as": "2019 Earnings",
        }, 
        {
            "name": "Earnings.2020",
            "as": "2020 Earnings",
        }, 
        {
            "name": "Earnings.2021",
            "as": "2021 Earnings",
        }, 
        {
            "name": "Earnings.2022",
            "as": "2022 Earnings",
        }, 
        {
            "name": "Earnings.2023",
            "as": "2023 Earnings",
        }, 
        {
            "name": "Earnings.2012",
            "as": "2012 Earnings",
        }, 
        {
            "name": "Earnings.2013",
            "as": "2013 Earnings",
        }, 
        {
            "name": "Earnings.2014",
            "as": "2014 Earnings",
        }, 
        {
            "name": "Earnings.2015",
            "as": "2015 Earnings",
        }, 
        {
            "name": "Earnings.2016",
            "as": "2016 Earnings",
        }, 
        {
            "name": "Earnings.2017",
            "as": "2017 Earnings",
        }, 
        {
            "name": "Earnings.2018",
            "as": "2018 Earnings",
        }, 
        {
            "name": "Earnings.2019",
            "as": "2019 Earnings",
        }, 
        {
            "name": "Earnings.2020",
            "as": "2020 Earnings",
        }, 
        {
            "name": "Earnings.2021",
            "as": "2021 Earnings",
        }, 
        {
            "name": "Earnings.2022",
            "as": "2022 Earnings",
        }, 
        {
            "name": "Earnings.2023",
            "as": "2023 Earnings",
        }, 
        {
            "name": "Earnings.2013",
            "as": "2013 Earnings",
        }, 
        {
            "name": "Earnings.2014",
            "as": "2014 Earnings",
        }, 
        {
            "name": "Earnings.2015",
            "as": "2015 Earnings",
        }, 
        {
            "name": "Earnings.2016",
            "as": "2016 Earnings",
        }, 
        {
            "name": "Earnings.2017",
            "as": "2017 Earnings",
        }, 
        {
            "name": "Earnings.2018",
            "as": "2018 Earnings",
        }, 
        {
            "name": "Earnings.2019",
            "as": "2019 Earnings",
        }, 
        {
            "name": "Earnings.2020",
            "as": "2020 Earnings",
        }, 
        {
            "name": "Earnings.2021",
            "as": "2021 Earnings",
        }, 
        {
            "name": "Earnings.2022",
            "as": "2022 Earnings",
        }, 
        {
            "name": "Earnings.2023",
            "as": "2023 Earnings",
        }, 
    ],

    "constraints": [
        {
            "dimensionName": "Area",
            "map": {
                "04000US26": ["26"], # Michigan
                "05000US26163": ["26163"], # Wayne County
                "05000US26125": ["26125"], # Oakland County
                "05000US26099": ["26099"], # Macomb
                "05000US26161": ["26161"], # Washtenaw
                "31000US19820": ["MSA19820"], # Detroit-Warren-Dearborn MSA
                **DETROIT_ZIPS
            }
        },
        { 
            "dimensionName": "Industry", 
            "map": { 
                "54": ["54"],
            },
        } 
    ]
}


response = requests.request("post", url, headers=headers, json=payload)


In [51]:
from detroit_tech_employment.throttle import throttle


auth = refresh_auth()
headers = {
    "Authorization": f"Bearer {auth['access_token']}",
    "Content-Type": "application/json"
}


@throttle(0.5)
def pull_metadata(dataset: str):

    url = f"https://agnitio.emsicloud.com/meta/dataset/{dataset}/2024.4"
    response = requests.request("get", url, headers=headers)

    try:
        meta = response.json()

        with open(f"lmi_datasets__{dataset}.json", "w") as f:
            json.dump(meta, f, indent=4)

    except json.JSONDecodeError as e:
        print(response.content)


In [52]:
with open("lmi_datasets.json") as f:
    datasets = json.load(f)


# for dataset in datasets["datasets"]:
    # pull_metadata(dataset["name"])

In [61]:
for dataset in datasets["datasets"]:
    with open(f"lmi_datasets__{dataset['name']}.json") as f:
        dictionary = json.load(f)

    print(dataset["name"], end=": ")
    found = False
    for item in dictionary["dimensions"]:
        if item["name"] == "Area":
            print(max(item["levelsStored"]))
            found = True
        
    if not found:
        print("No AREA")

emsi.us.ind.firm: 3
emsi.us.staffing: 4
emsi.us.staffing.earn: No AREA
emsi.us.occ.hires.seps: 3
emsi.us.industry: 4
emsi.us.industry.raceethnicity: 3
emsi.us.occupation: 4
emsi.us.occupation.detailed: 3
emsi.us.occupation.raceethnicity: 3
emsi.us.unemployment.occupation: 3


In [90]:
url = "https://agnitio.emsicloud.com/emsi.us.staffing/2024.2"

payload = {
    "metrics": [
        {
            "name": "Jobs.2001"
        },
        {
            "name": "Jobs.2002"
        },
        {
            "name": "Jobs.2003"
        },
        {
            "name": "Jobs.2004"
        },
        {
            "name": "Jobs.2005"
        },
        {
            "name": "Jobs.2006"
        },
        {
            "name": "Jobs.2007"
        },
        {
            "name": "Jobs.2008"
        },
        {
            "name": "Jobs.2009"
        },
        {
            "name": "Jobs.2010"
        },
        {
            "name": "Jobs.2011"
        },
        {
            "name": "Jobs.2012"
        },
        {
            "name": "Jobs.2013"
        },
        {
            "name": "Jobs.2014"
        },
        {
            "name": "Jobs.2015"
        },
        {
            "name": "Jobs.2016"
        },
        {
            "name": "Jobs.2017"
        },
        {
            "name": "Jobs.2018"
        },
        {
            "name": "Jobs.2019"
        },
        {
            "name": "Jobs.2020"
        },
        {
            "name": "Jobs.2021"
        },
        {
            "name": "Jobs.2022"
        },
        {
            "name": "Jobs.2023"
        },
        {
            "name": "Jobs.2024"
        },
        {
            "name": "Jobs.2025"
        },
        {
            "name": "Jobs.2026"
        },
        {
            "name": "Jobs.2027"
        },
        {
            "name": "Jobs.2028"
        },
        {
            "name": "Jobs.2029"
        },
        {
            "name": "Jobs.2030"
        },
        {
            "name": "Jobs.2031"
        },
        {
            "name": "Jobs.2032"
        },
        {
            "name": "Jobs.2033"
        },
        {
            "name": "Jobs.2034"
        }
    ],

    "constraints": [
        {
            "dimensionName": "Area",
            "map": {
                "04000US26": ["26"], # Michigan
                "05000US26163": ["26163"], # Wayne County
                "05000US26125": ["26125"], # Oakland County
                "05000US26099": ["26099"], # Macomb
                "05000US26161": ["26161"], # Washtenaw
                "31000US19820": ["MSA19820"], # Detroit-Warren-Dearborn MSA
                **DETROIT_ZIPS
            }
        },
        { 
            "dimensionName": "Industry", 
            "map": { 
                "54": ["54"],
            },
        },
        { 
            "dimensionName": "Occupation", 
            "map": { 
                "15-1250": ["15-1250"],
            },
        },
    ]
}

response = requests.request("post", url, headers=headers, json=payload)

In [91]:
response.content

b'{"data":[{"name":"Area","type":"String","rows":["04000US26","04000US26","05000US26099","05000US26099","05000US26125","05000US26125","05000US26161","05000US26161","05000US26163","05000US26163","31000US19820","31000US19820","86000US48201","86000US48201","86000US48202","86000US48202","86000US48203","86000US48203","86000US48204","86000US48204","86000US48205","86000US48205","86000US48206","86000US48206","86000US48207","86000US48207","86000US48208","86000US48208","86000US48209","86000US48209","86000US48210","86000US48210","86000US48211","86000US48211","86000US48212","86000US48212","86000US48213","86000US48213","86000US48214","86000US48214","86000US48215","86000US48215","86000US48216","86000US48216","86000US48217","86000US48217","86000US48219","86000US48219","86000US48221","86000US48221","86000US48223","86000US48223","86000US48224","86000US48224","86000US48226","86000US48226","86000US48227","86000US48227","86000US48228","86000US48228","86000US48233","86000US48233","86000US48234","86000US482

In [92]:
df = convert_to_df(response.json())

In [94]:
df[df["Area"] == "86000US48202"]

,Area,Industry,Occupation,Jobs.2001,Jobs.2002,Jobs.2003,Jobs.2004,Jobs.2005,Jobs.2006,Jobs.2007,...,Jobs.2025,Jobs.2026,Jobs.2027,Jobs.2028,Jobs.2029,Jobs.2030,Jobs.2031,Jobs.2032,Jobs.2033,Jobs.2034
14,86000US48202,54,15-1252,47.039703,28.057332,29.515000,29.248400,35.674152,32.267524,27.965837,...,78.229973,78.251238,78.219277,78.531873,78.547279,78.561668,78.568295,78.595896,78.559450,78.520337
15,86000US48202,54,15-1253,3.069625,1.764017,1.853437,1.816469,2.235683,2.037119,1.838422,...,4.856266,4.807874,4.771210,4.738516,4.690607,4.645554,4.601970,4.562436,4.519593,4.477547
